# Read PDF information with LLMs
### let's test how good it the LLMs at extracting information from pdfs

In [110]:
packages = ['pymongo', 'gridfs', 'python-dotenv', 'openai']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
from gridfs import GridFS
from dotenv import load_dotenv
from openai import OpenAI

endpoint = '10.0.10.5:27017'
#endpoint = '192.168.1.10:27017'
database = 'supercias'
collection = 'companies'
pdfs_path = '../storage/pdfs/'
# make dir path if it does not exist
if not os.path.exists(pdfs_path):
    os.makedirs(pdfs_path)

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
collection = db[collection]

#find pdf file
comp = collection.find_one()
print(comp['name'])


docs = comp['Documentos online']['downloaded']
pdf_names = []
#if company has documents
for value in docs.values():
    if isinstance(value, list) and len(value) > 0:
        [ pdf_names.append(v) for v in value ]


Defaulting to user installation because normal site-packages is not writeable


ServerSelectionTimeoutError: 10.0.10.5:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66d68cacf0d5a82ff9644993, topology_type: Unknown, servers: [<ServerDescription ('10.0.10.5', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('10.0.10.5:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

Download an pdf file to make sure that it is working

In [106]:
pdf_filename = pdf_names[1]

def write_to_disk(filename):
    print(f'fetching {filename} to disk...')
    # get pdf from GridFS
    fs = GridFS(db, collection='companies')
    pdf  = fs.find({'filename': filename})
    try:
        pdf = next(pdf)
    except StopIteration:
        print(f'File not found in GridFS')
        return
    # write file to disk
    with open(pdfs_path+filename, 'wb') as f:
        f.write(pdf.read())


def delete_from_disk(filename):
    print(f'deleting {filename} from disk...')
    try:
        os.remove(pdfs_path+filename)
    except FileNotFoundError:
        print(f'File {filename} not found in disk')
        return

write_to_disk(pdf_filename)

fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1996-09-06.pdf to disk...
File not found in GridFS


Make sure that the the openai api is workig correcly

In [94]:
# Load the .env file
load_dotenv()
# Example usage
api_key = os.getenv('OPENAI_KEY')
# Initialize the OpenAI client
client = OpenAI(
    api_key=api_key,
)

def prompt_openai(prompt):

    # Create a chat completion
    chat_completion = client.chat.completions.create(
        messages=[ prompt ],
        model="gpt-3.5-turbo",
    )

    return chat_completion.choices[0].message.content
print(prompt_openai({ "role": "system", "content": "is this working correcly?" }))

Yes, it appears to be working correctly. How can I assist you further?


uploading the pdf dirrecly to openai is not possible. it is not able to extract the data as of today (2024-09-02). 

So we need to extract the text first.

In [98]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import io

def extract_text_from_image_pdf(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    text = ""

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        
        # Extract images from the page
        image_list = page.get_images(full=True)
        
        # If there are images, use OCR to extract text
        if image_list:
            for image_index, img in enumerate(page.get_images(full=True)):
                xref = img[0]
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]

                # Convert to a PIL Image
                image = Image.open(io.BytesIO(image_bytes))
                
                # Use pytesseract to do OCR on the image
                text += pytesseract.image_to_string(image)
        else:
            # If no images, extract text as usual
            text += page.get_text()

    return text

pdf_path = pdfs_path+pdf_filename
extracted_text = extract_text_from_image_pdf(pdf_path)
print(extracted_text)

€¢

REFERENOTA

GISIS S.A. % 669 @ Nm 9
Duran, 11 de octubre de 1.999
\ 1
Sefior \
JUAN XAVIER CORDOVEZ ORTEGA “x

Ciudad.-

De mis consideraciones:

Por medio de la presente me es grato informarle que la Junta General Extraordinaria de
ja compafiia GISIS S. A. en sesion celebrada el dia de hoy resolvio reelegirlo
PRESIDENTE de la compafiia por el lapso estatutario de CINCO afios mas.

En el ejercicio de su cargo le correspondera ejercer individualmente la
representacion legal de la compafiia, tanto judicial como extrajudicialmente.

Los estatutos sociales de la compafiia constan en la escritura publica otorgada ante el
Notario Vigésimo Tercero del canton Guayaquil, abogado Mario Baquerizo Gonzalez,
el 8 de agosto de 1.994, la misma que ha sido inscrita en el Registro Mercantil del
mismo canton el | de septiembre de 1.994.

na,

ve Ferreyros Ce,

GERENTE GENERAL

ACEPTACION: Yo, Juan Xavier Cordovez Ortega, portador de fa cédula de identidad No.
090582809-1, de nacionalidad ecuatoriana

Now ask the summerize correcly the infomation extracted from the extracted text

In [108]:
prompt = "Give me all the information you can aquire from this text. ive it in json file " + pdf_filename + ": \n"+  extracted_text,
response = prompt_openai({ "role": "system", 
    "content": prompt,
})
print(response)

{
    "company": {
        "name": "GISIS S.A.",
        "registration_number": "669",
        "date": "11 de octubre de 1.999",
        "address": "Duran",
        "city": "Guayaquil",
        "legal_representative": "Juan Xavier Cordovez Ortega",
        "general_meeting_decision": "Reelection of Juan Xavier Cordovez Ortega as President for five more years",
        "statutes": {
            "public_deed_date": "8 de agosto de 1.994",
            "notary": "Mario Baquerizo Gonzalez",
            "registration_date": "1 de septiembre de 1.994"
        }
    },
    "roles": {
        "president": {
            "name": "Juan Xavier Cordovez Ortega",
            "nationality": "ecuadorian",
            "id_number": "090582809-1",
            "address": "Guayaquil, kilómetro 6/4 de la vía Duran - Tambo",
            "acceptance_date": "11 de octubre de 1.999"
        },
        "general_manager": {
            "name": "Jose Ferreyros Ce"
        }
    },
    "acceptance_document": {
     

now let iterate trhough every document to see if we can get a infomration fo the company as a whole

In [109]:
data = []
# for every pdf file in company
for pdf in pdf_names:
    # read the file
    written = write_to_disk(pdf)
    if(written):
        # extract text
        text = extract_text_from_image_pdf(pdf)
        # intepret the data with the llm
        response = prompt_openai({ "role": "system", 
            "content": "Give me all the information you can aquire from this text. give it in json file " + pdf + ": \n" + text,
        })
        print(response)
        # save the data
        data.push({ 'filename': pdf, 'text_extracted': text, 'interpretation': response })
        
print(data)




fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1994-09-13.pdf to disk...


ServerSelectionTimeoutError: 192.168.1.10:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66d607dcf0d5a82ff9644992, topology_type: Single, servers: [<ServerDescription ('192.168.1.10', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('192.168.1.10:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>